In [2]:
import os
import pandas as pd
import numpy as np

# sub_df = pd.read_csv('sub_data/all.csv')
vol_df = pd.read_csv('VolumeVoxel_df.csv')
# only CN and MCI spreadsheets have the image data id necessary to make the map
sub_df = pd.read_csv('sub_data/data_list_SCZ_AD_MCI_CN (1).xlsx - CN_All.csv').append(pd.read_csv('sub_data/data_list_SCZ_AD_MCI_CN (1).xlsx - MCI_All.csv'))

In [3]:
# extract just the problematic portion
# sub_df = sub_df.iloc[448:]
vol_df = vol_df.iloc[:423]

In [ ]:
sub_df

In [6]:
# now I need to somehow build a map between rows... oh god
# loop through set of the truncated ids
truncIDs = set(sub_df['subject'])
truncIDs

{'002_S_1070',
 '002_S_1261',
 '002_S_1268',
 '002_S_1280',
 '005_S_0324',
 '005_S_0448',
 '005_S_0553',
 '005_S_0572',
 '007_S_1206',
 '007_S_1222',
 '012_S_1009',
 '016_S_1117',
 '016_S_1121',
 '016_S_1326',
 '018_S_0369',
 '018_S_0425',
 '018_S_0450',
 '023_S_0030',
 '023_S_0031',
 '023_S_0058',
 '023_S_0061',
 '023_S_0331',
 '023_S_0376',
 '023_S_0388',
 '023_S_0604',
 '023_S_0625',
 '023_S_0926',
 '023_S_0963',
 '023_S_1046',
 '023_S_1126',
 '023_S_1190',
 '023_S_1247',
 '027_S_0307',
 '027_S_0403',
 '027_S_0835',
 '027_S_1387',
 '031_S_0830',
 '031_S_1066',
 '032_S_0677',
 '032_S_1169',
 '037_S_0303',
 '037_S_0501',
 '051_S_1123',
 '051_S_1331',
 '053_S_0507',
 '067_S_0607',
 '100_S_0015',
 '100_S_0190',
 '100_S_1286',
 '116_S_0382',
 '116_S_0649',
 '116_S_1232',
 '116_S_1249',
 '127_S_0260',
 '127_S_0622',
 '130_S_0449',
 '130_S_0505',
 '130_S_0886',
 '130_S_0969',
 '136_S_0086',
 '136_S_0184',
 '136_S_0195',
 '136_S_0196',
 '136_S_0429',
 '136_S_0579',
 '136_S_1227'}

In [20]:
# all we care about in the vol df is the id
vol_ids = list(vol_df['filename'])

In [35]:
sub_df[sub_df.subject == '136_S_1227'].to_numpy()[:,5]

array([6, 5, 4, 2, 3], dtype=object)

In [46]:
godmap = {}
for truncID in truncIDs:
    print(truncID)
    sub_rows = sub_df[sub_df.subject == truncID]
    vids = [vid for vid in vol_ids if truncID in vid]
    if not len(sub_rows) == len(vids):
        print('WAT')
    # find coreespondence
    # first sort rows by visit, then by image id
    sorted_subs = np.array(sorted(sub_rows.to_numpy(), key=lambda row: row[5]))
    # if there are no duplicate visits, we can proceed
    if len(set(sorted_subs[:,5])) == len(sorted_subs):
        sorted_vids = sorted(vids, key=lambda vid: vid[-1])
        if not all([vid[-3] == '1' for vid in sorted_vids]):
            print('bad!')
            # pass


    # break

007_S_1222
037_S_0303
130_S_0449
bad!
136_S_0195
002_S_1268
bad!
032_S_1169
bad!
005_S_0324
bad!
031_S_0830
bad!
051_S_1123
bad!
005_S_0553
100_S_0015
bad!
130_S_0505
136_S_0196
016_S_1121
bad!
016_S_1117
007_S_1206
136_S_0429
100_S_0190
027_S_0835
002_S_1070
bad!
012_S_1009
023_S_1247
018_S_0369
023_S_0926
100_S_1286
bad!
023_S_0625
136_S_0579
018_S_0450
bad!
127_S_0260
037_S_0501
bad!
136_S_0184
023_S_0331
002_S_1261
031_S_1066
bad!
053_S_0507
027_S_0307
bad!
023_S_0376
130_S_0969
002_S_1280
023_S_0061
116_S_1232
bad!
023_S_0388
023_S_0030
016_S_1326
023_S_1046
130_S_0886
bad!
023_S_0031
027_S_1387
bad!
023_S_0604
005_S_0448
bad!
116_S_0649
051_S_1331
bad!
023_S_1126
116_S_1249
bad!
067_S_0607
WAT
023_S_0058
023_S_0963
032_S_0677
136_S_0086
005_S_0572
bad!
127_S_0622
018_S_0425
023_S_1190
116_S_0382
027_S_0403
bad!
136_S_1227


In [ ]:
raise Exception('stop')

In [ ]:
merged1 = pd.merge(sub_df, vol_df, how='left', on='filename')

In [ ]:
merged1

In [ ]:
# list(merged1['class'] == merged1['Target'])[:423]

In [ ]:
merged1.to_csv('merged1.csv')

In [ ]:
sub_id_col = vol_df['filename'].iloc[:423].apply(get_id)
vol_df['subject'] = sub_id_col

In [ ]:
vol_df

In [ ]:
sub_df

In [ ]:
merged2 = pd.merge(sub_df, vol_df, how='inner', on='subject')

In [ ]:
# list(merged2.columns)

In [ ]:
fff = merged2.drop_duplicates('filename_y')

In [ ]:
all(fff['class'] == fff['Target'])

In [ ]:
##########################

In [ ]:
worked = pd.merge(sub_df, vol_df_bottom, on='filename', how='left')

In [ ]:
worked[:448]

In [ ]:
# worked.to_csv('merged.csv')

In [ ]:
worked_top = worked.iloc[:448].copy(deep=True) #merge completed
worked_bottom = worked.iloc[448:].copy(deep=True)

In [ ]:
worked_bottom

In [ ]:
worked_bottom = pd.merge(worked_bottom, vol_df_top, how='outer', on='subject')

In [ ]:
worked_bottom

In [ ]:
worked = pd.merge(worked, vol_df_top, on='subject', how='left')

In [ ]:
worked.to_csv('merged.csv')